In [1]:
import glob
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / N

In [22]:
def getSentimentTS(city, fileList=None):
    if(fileList is None):
        fileList = sorted(glob.glob(f"./Datasets/cities-sentiment/{city}/*"))
    sentimentList = []
    dates = []
    M = []
    D = []
    Y = []
    for idx, file in enumerate(fileList):
        clear_output(wait=True)
#         print(f"{idx}/{len(fileList)}")
        df = pd.read_csv(file)
        date = file.split("/")[-1].split(".")[0]
        y, m, d = date.split("-")
        Y.append(int(y))
        M.append(int(m))
        D.append(int(d))
        dates.append(date)
        scores = df[["negative", "positive", "neutral"]]
        sentimentCount = dict(pd.DataFrame(scores.idxmax(axis=1)).groupby(0).size())
        sentimentList.append(sentimentCount)
    qf = pd.DataFrame(sentimentList)
    qf["sentimentIndex"] = (qf["positive"]*1)+(-1*qf["negative"])
    qf["sentimentRatio"] = qf["positive"]/qf["negative"]
    qf["total"] = qf["positive"]+qf["negative"]+qf["neutral"]
    qf["date"] = dates
    qf["Y"] = Y
    qf["M"] = M
    qf["D"] = D
#     rm = running_mean(list(qf["sentimentRatio"]), 30)
    return qf

In [23]:
def getPrecovid(city):
    return getSentimentTS(city, fileList=sorted(glob.glob(f"./Datasets/cities-sentiment/{city}/*"))[:365+27])

In [24]:
def getPostcovid(city):
    return getSentimentTS(city, fileList=sorted(glob.glob(f"./Datasets/cities-sentiment/{city}/*"))[365+27:])

In [28]:
def getMonthlyIndex(df):
    newDF = df.groupby(["Y", "M"]).sum()
    newDF["sentimentRatio"] = newDF["positive"]/newDF["negative"]
    return newDF

In [36]:
# TODO
# 1. generate city level datasets pre and post

In [37]:
cityNames = ["Bangalore", "Chennai", "Kolkata", "Hyderabad", "Delhi", "Mumbai"]

In [44]:
for city in cityNames:
    pre = getMonthlyIndex(getPrecovid(city))
    post = getMonthlyIndex(getPostcovid(city))
    pre.to_csv(f"./cleaned_datasets/sentimentIndex/pre/{city}.csv")
    post.to_csv(f"./cleaned_datasets/sentimentIndex/post/{city}.csv")